In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [7]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions= "You are a helpful assistant. Respond only in caps.",
    model = 'gpt-4o-mini'
)

result = await Runner.run(agent, "Hello! How are you?")
result.final_output

"HELLO! I'M JUST A PROGRAM, BUT I'M READY TO HELP YOU! HOW CAN I ASSIST YOU TODAY?"

In [11]:
joke_agent = Agent(
    name = "Joke Agent",
    instructions= "You are a joke teller. You are given a topic and you need to tell a joke about it."
)

topic = "History"
result = await Runner.run(joke_agent, topic)
result.final_output

'Why was the history teacher always so calm?\n\nBecause she had a lot of dates to fall back on!'

In [13]:
language_agent = Agent(
    name = "Language Agent",
    instructions= "You are a language expert. You are given a joke and you need to rewrite it in a different language."
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to Italy: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

Original joke:
Why did the historian break up with the librarian?

Too much drama—he wanted more of a non-friction relationship!

Translated joke:
Perché lo storico ha lasciato il bibliotecario?

Troppi drammi—voleva una relazione più "non-fiction"!
